# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

In [2]:
KEYWORDS = ['python', 'парсинг']

In [3]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')


In [35]:
links = pd.DataFrame()
for post in posts:
#     post.text содержит в себе всю текстовую preview информацию, включая заголовок, список хабов и собственно начало статьи
#     будем считать, что если post.text содержит хотя бы одно из ключевых слов, то этот пост должен попасть в нашу выборку

    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    if any([keyword in post.text.lower() for keyword in KEYWORDS]):
        title_element = post.find('a', class_='post__title_link')
        post_time = post.find('span', class_='post__time')
#         print(post_time.text, title_element.text)
        row = {'date':post_time.text, 'title':title_element.text, 'link':title_element.attrs.get('href')}
        links = pd.concat([links, pd.DataFrame([row])], ignore_index=True)


In [36]:
links 

,date,title,link
0,вчера в 22:02,Как принять сигналы немецкого ВМФ с помощью зв...,https://habr.com/ru/post/526824/
1,вчера в 18:42,Хост KVM в паре строчек кода,https://habr.com/ru/company/timeweb/blog/526818/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [37]:
links_full = pd.DataFrame()

# Зайдем с другой стороны. Раз нам нужно анализировать содержания статей, а не только превью, 
# соберем не список постов главной страницы, а список ссылок на посты, которые на ней есть.

title_links = soup.find_all('a', class_='post__title_link')
for title_link in title_links:
    link = title_link.attrs.get('href')
    inner_soup = BeautifulSoup(requests.get(link).text, 'html.parser')
#     полный текст статьи, а также список тегов, хабов и пр. можно вытащить из блока post__wrapper 
    full_post = inner_soup.find('div', class_='post__wrapper')
#     print(full_post.text)
    if any([keyword in full_post.text.lower() for keyword in KEYWORDS]):
#         заголовок и ссылка у нас уже есть, нужно вытащить дату и текст статьи
        post_time = full_post.find('span', class_='post__time')
        post_time = post_time.attrs.get('data-time_published')
        
        post_content = full_post.find('div', class_ = 'post__text').text
        
        row = {'date':post_time, 'title':title_link.text, 'link':link, 'content':post_content}
        links_full = pd.concat([links_full, pd.DataFrame([row])], ignore_index=True)
        

links_full['date'] = pd.to_datetime(links_full['date'], format='%Y-%m-%dT%H:%MZ')        
links_full    

,date,title,link,content
0,2020-11-06 19:36:00,AI на минималках: пишем свой Сокобан и учим ко...,https://habr.com/ru/post/526842/,\nВ этой статье я расскажу как написать свою р...
1,2020-11-06 19:02:00,Как принять сигналы немецкого ВМФ с помощью зв...,https://habr.com/ru/post/526824/,"Привет, Хабр.\n\r\nТема приема и анализа сверх..."
2,2020-11-06 15:42:00,Хост KVM в паре строчек кода,https://habr.com/ru/company/timeweb/blog/526818/,"Привет!\n\r\nСегодня публикуем статью о том, к..."
3,2020-11-06 15:31:00,"13 игр, чтобы поиграть, взломать и убить ими э...",https://habr.com/ru/company/skillfactory/blog/...,В конце ноября у нас стартует новый поток курс...


In [ ]:
# интересный момент связан с получением даты статьи. На заглавной странице хабра даты текущего и вчерашнего дня 
# выводятся в описательном формате (сегодня в 12:27, вчера в 14:40), нет каких-то указаний на полный формат даты
# с выводом года, месяца и дня. Логично было бы преобразовать это описание в более содержательный формат.
# Мы можем это сделать, зайдя по ссылке в соответствующую статью. Там у такого же тега span с классом 'post__time'
# есть еще и атрибут 'data-time_published', в котором дата хранится полностью

In [38]:
def get_post_time(link):
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    post_time = soup.find('span', class_='post__time')
    return post_time.attrs.get('data-time_published')
    

In [39]:
# применим функцию к датасету из первой части задания
links.loc[:, 'date'] = links['link'].apply(get_post_time)
links

,date,title,link
0,2020-11-06T19:02Z,Как принять сигналы немецкого ВМФ с помощью зв...,https://habr.com/ru/post/526824/
1,2020-11-06T15:42Z,Хост KVM в паре строчек кода,https://habr.com/ru/company/timeweb/blog/526818/


In [40]:
# можно преобразовать это значение в тип данных datetime и в дальнейшем работать с этим столбцом, как с нормальными датами
links['date'] = pd.to_datetime(links['date'], format='%Y-%m-%dT%H:%MZ')
links

,date,title,link
0,2020-11-06 19:02:00,Как принять сигналы немецкого ВМФ с помощью зв...,https://habr.com/ru/post/526824/
1,2020-11-06 15:42:00,Хост KVM в паре строчек кода,https://habr.com/ru/company/timeweb/blog/526818/


In [41]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2 non-null      datetime64[ns]
 1   title   2 non-null      object        
 2   link    2 non-null      object        
dtypes: datetime64[ns](1), object(2)
memory usage: 176.0+ bytes


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [42]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
data = {"emailAddresses":["xxx@x.ru"]}
headers = {
         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36",
         "Vaar-Header-App-Product": "hackcheck-web-avast",
         "Vaar-Version": "0"
}
req = requests.post(url, data=json.dumps(data), headers = headers)
req

<Response [200]>

In [43]:
print(req.json())

{'breaches': {'3176': {'breachId': 3176, 'site': 'parapa.mail.ru', 'recordsCount': 5029003, 'description': "In July and August 2016, two criminals executed attacks against three separate forums hosted by Mail.ru including the Russian forum Parapa. Shortly after the breach occurred, the contents of Parapa's database were leaked publicly. The database contains usernames, email addresses, and hashed passwords for around 5 million users.", 'publishDate': '2017-02-14T00:00:00Z', 'statistics': {'usernames': 5029000, 'passwords': 5029003, 'emails': 4941344}}, '12': {'breachId': 12, 'site': 'vk.com', 'recordsCount': 91262655, 'description': "Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a pr

In [44]:
# посмотрим, что мы получили в результате запроса.
# видим, что чтобы достать необходимую нам инфу, надо разбирать значения по ключу breaches  

for k, v in req.json().items():
    print(k)
    print('----------')
    print(v)
    print()


breaches
----------
{'3176': {'breachId': 3176, 'site': 'parapa.mail.ru', 'recordsCount': 5029003, 'description': "In July and August 2016, two criminals executed attacks against three separate forums hosted by Mail.ru including the Russian forum Parapa. Shortly after the breach occurred, the contents of Parapa's database were leaked publicly. The database contains usernames, email addresses, and hashed passwords for around 5 million users.", 'publishDate': '2017-02-14T00:00:00Z', 'statistics': {'usernames': 5029000, 'passwords': 5029003, 'emails': 4941344}}, '12': {'breachId': 12, 'site': 'vk.com', 'recordsCount': 91262655, 'description': "Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 f

In [26]:
# посмотрим на breaches
for k, v in req.json()['breaches'].items():
    print(k)
    print('----------')
    print(v)
    print()
    

3176
----------
{'breachId': 3176, 'site': 'parapa.mail.ru', 'recordsCount': 5029003, 'description': "In July and August 2016, two criminals executed attacks against three separate forums hosted by Mail.ru including the Russian forum Parapa. Shortly after the breach occurred, the contents of Parapa's database were leaked publicly. The database contains usernames, email addresses, and hashed passwords for around 5 million users.", 'publishDate': '2017-02-14T00:00:00Z', 'statistics': {'usernames': 5029000, 'passwords': 5029003, 'emails': 4941344}}

12
----------
{'breachId': 12, 'site': 'vk.com', 'recordsCount': 91262655, 'description': "Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a 

In [49]:
# отсюда уже можно доставать наши данные для датасета.'site' - источник утечки, 'description' - описание,
# 'publishDate' - дата утечки
# теперь можно собрать весь код для нашего списка почтовых адресов

EMAILS = ["xxx@x.ru", "yyy@y.com"]
df = pd.DataFrame()
for email in EMAILS:
    url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    data = {"emailAddresses":[email]}
    headers = {
             "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36",
             "Vaar-Header-App-Product": "hackcheck-web-avast",
             "Vaar-Version": "0"
    }
    req = requests.post(url, data=json.dumps(data), headers = headers)
    for details in req.json()['breaches'].values():
        row = {
            'email':email,
            'breach_date': details['publishDate'],
            'breach_source': details['site'],
            'description': details['description']
        }
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

df['breach_date'] = pd.to_datetime(df['breach_date'], format='%Y-%m-%dT%H:%M:%SZ')
df
    

,email,breach_date,breach_source,description
0,xxx@x.ru,2019-03-28,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2017-02-14,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
6,xxx@x.ru,2017-01-31,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
7,xxx@x.ru,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
8,yyy@y.com,2019-03-28,verifications.io,Big data e-mail verification platform verifica...
9,yyy@y.com,2019-02-21,www.dangdang.com,"This is a list of email addresses only, and as..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [62]:
REQUEST = 'https://api.vk.com/method/wall.get?'
TOKEN = ''
VERSION = '5.103'
SLEEP = 0.33

In [63]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': 'netology',
    'count': 50
}

In [64]:
res = requests.get(REQUEST, params)
res

<Response [200]>

In [65]:
res.json()

{'response': {'count': 7320,
  'items': [{'id': 57385,
    'from_id': -30159897,
    'owner_id': -30159897,
    'date': 1604647980,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Точно решили, кем хотите стать в диджитале, но не знаете с чего начать? \nСобрали для вас подборку курсов и лекций, которые помогут сделать первые шаги в профессии: \n\n🔹 9 ноября стартует бесплатный курс «Как стать программистом» \nУзнаем, чего ждать от профессии, какие востребованные сферы разработки существуют и с чего начать новичку → http://netolo.gy/fYe \n \n🔹 17 ноября — старт бесплатного курса «Data Science: будущее для каждого» \nРазбираемся, какие навыки развивать в первую очередь, если вы хотите начать карьеру в Data Science, какие направления сферы работы с данными существуют и с чего лучше начать → http://netolo.gy/fYf \n\n🔹 23 ноября — бесплатный курс «Как стать продакт- или проджект-менеджером» \nПоговорим, какие специалисты работают с продуктом на разных этапах, какие навыки необ

In [67]:
vk_wall = pd.DataFrame(res.json()['response']['items'], columns=['date', 'text'])
vk_wall.head()

,date,text
0,1604647980,"Точно решили, кем хотите стать в диджитале, но..."
1,1604586780,Почему за некоторые задачи так сложно взяться?...
2,1604565660,"Чтобы сделать приличный сайт, не обязательно б..."
3,1604499480,Выходной в середине недели — самое время устро...
4,1604474580,Чтобы создавать уникальные дизайнерские продук...


In [76]:
vk_wall['date'] = pd.to_datetime(vk_wall['date'],unit='s')
vk_wall.head()

,date,text
0,2020-11-06 07:33:03,"Точно решили, кем хотите стать в диджитале, но..."
1,2020-11-05 14:33:03,Почему за некоторые задачи так сложно взяться?...
2,2020-11-05 08:41:03,"Чтобы сделать приличный сайт, не обязательно б..."
3,2020-11-04 14:18:03,Выходной в середине недели — самое время устро...
4,2020-11-04 07:23:03,Чтобы создавать уникальные дизайнерские продук...


In [94]:
vk_wall.shape

(50, 2)

In [ ]:
# выглядит как нужно, и длина датасета правильная. Однако если мы сравним даты в нашем датасете, полученные из unixtime,
# и даты постов на страничке группы, увидим, что на страничке группы все записи смещены на 3 часа вперед 
# относительно дат из датасета. Московское время как раз соответствует UTC+3, и при необходимости мы можем дополнительно
# преобразовать даты с учетом разницы между локальным временем и UTC

In [90]:
from dateutil import tz

def utc_to_local(utc_dt):
    HERE = tz.tzlocal()
    UTC = tz.gettz('UTC')
    return utc_dt.replace(tzinfo=UTC).astimezone(HERE)

In [91]:
vk_wall['date'] = vk_wall['date'].apply(utc_to_local)
vk_wall.head()
# теперь даты совпадают

,date,text
0,2020-11-06 10:33:03+03:00,"Точно решили, кем хотите стать в диджитале, но..."
1,2020-11-05 17:33:03+03:00,Почему за некоторые задачи так сложно взяться?...
2,2020-11-05 11:41:03+03:00,"Чтобы сделать приличный сайт, не обязательно б..."
3,2020-11-04 17:18:03+03:00,Выходной в середине недели — самое время устро...
4,2020-11-04 10:23:03+03:00,Чтобы создавать уникальные дизайнерские продук...


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
